# UPDATE: After contacting QM, found out that in version 0.8, opening multiple QMs like this is not possible. But, it will be possible with version 0.9, which comes out in a few weeks.

### General goal: To investigate the behaviour of the QMM API wrt opening and closing quantum machines.

#### Specific goal: Find the conditions for interacting with multiple quantum machines at the same time. Use case - if I have 2 samples (and thus two distinct Quantum Machines), I want to be able to play pulses with the same OPX to/from each sample independently of the other. Can I do that???

Related qn - in the image on this page from the docu - http://qm-docs.s3.amazonaws.com/v0.8/python/concept_overview/multi_machine.html - does the OPX refer to a unique OPX or different resources (e.g. ports) on the same OPX?

In [6]:
# ensure relative imports are correct
import sys
sys.path.insert(0, '../..') # this notebook resides in a sub-sub-directory in the main package

In [7]:
from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.QuantumMachine import QuantumMachine
from qm.qua import *

from test_qmm_qm_config1 import config1
from test_qmm_qm_config2 import config2

##### Please restart your Jupyter notebook kernel to purge exisiting variables before beginning each attempt below. This is simply a precaution to ensure that namespace collisions do not cause unnecessary problems...

### Attempt 1. Opening multiple QMs with the same config

In [8]:
qmm1 = QuantumMachinesManager()

2021-04-12 23:53:12,696 - qm - INFO - Performing health check
2021-04-12 23:53:12,699 - qm - INFO - Health check passed


In [4]:
qm1 = qmm1.open_qm(config1, close_other_machines=False)
qm1_id = qm1.id
print(qm1_id)

qm-1618294088016


In [12]:
qm2 = qmm1.open_qm(config1, close_other_machines=False)
qm2_id = qm2.id
print(qm2_id)

qm-1618293942258


In [13]:
print(qmm1.list_open_quantum_machines())

['qm-1618293942258']


In [15]:
# running this will throw a 'Machine does not exist' exception
#qm2 = qmm1.get_qm(qm1_id)

In [10]:
qmm1.version()

{'client': '0.80.627', 'server': '0.80.627'}

#### Result: only the last opened QM remains open, even after explicitly setting the close_other_machines flag to False. Opening a new QM closes all other QMs.

In [5]:
# try the same but open the second qm while the first one is executing a job
# run this block before opening the second qm
with program() as cw:
    with infinite_loop_():
        play('CW', 'qubit')
        play('CW', 'rr')
job1 = qm1.execute(cw)

# Result: opening the second qm stops job1. This happens if we open the qm on another pc too.

2021-04-12 23:08:12,709 - qm - INFO - Flags: 
2021-04-12 23:08:12,710 - qm - INFO - Executing high level program


### Attempt 2. Try opening multiple QMs with the same config, but each QM has a different QMM.

#### Note: as the QMs live on the server and the QMM merely establishes a connection to that server, I wouldn't expect this to work. But, let's try it anyway. 

In [10]:
qmm1 = QuantumMachinesManager()
qmm2 = QuantumMachinesManager()

2021-04-12 21:00:44,926 - qm - INFO - Performing health check
2021-04-12 21:00:44,928 - qm - INFO - Health check passed
2021-04-12 21:00:44,931 - qm - INFO - Performing health check
2021-04-12 21:00:44,933 - qm - INFO - Health check passed


In [11]:
qm1 = qmm1.open_qm(config1, close_other_machines=False)
qm1_id = qm1.id
print(qm1_id)

qm-1618286443659


In [12]:
qm2 = qmm2.open_qm(config1, close_other_machines=False)
qm2_id = qm2.id
print(qm2_id)

qm-1618286447571


In [13]:
print(qmm1.list_open_quantum_machines())

['qm-1618286447571']


In [14]:
print(qmm2.list_open_quantum_machines())

['qm-1618286447571']


#### Result: as expected, we get the same result as attempt 1. 

### Attempt 3. Use only one QMM. Open two QMs, each with their own controller spec defined in separate config dicts. 

#### Note: we ensure that the same OPX resources aren't double-used. E.g. in config 1, the qubit and RR use 'con1' ports AO 1, 2, 3, 4 and AI 1. In config 2, the qubit and RR use 'con1' ports AO 5, 6, 7, 8 and AI 2. Double-use does not throw an error, but we avoid it because it does not encapsulate our use case (see Specific goal in the first cell of this notebook). 

In [3]:
qmm1 = QuantumMachinesManager()

2021-04-12 21:14:14,100 - qm - INFO - Performing health check
2021-04-12 21:14:14,107 - qm - INFO - Health check passed


In [4]:
# in config 1, 'con1' ports AO 1, 2, 3, 4 and AI 1 are used
qm1 = qmm1.open_qm(config1, close_other_machines=False)
qm1_id = qm1.id
print(qm1_id)

qm-1618287252745


In [5]:
# in config 1, 'con2' ports AO 5, 6, 7, 8 and AI 2 are used
qm2 = qmm1.open_qm(config2, close_other_machines=False)
qm2_id = qm2.id
print(qm2_id)

qm-1618287253980


In [6]:
print(qmm1.list_open_quantum_machines())

['qm-1618287253980']


#### Result: we get the same result as attempts 1 and 2. 

### Attempt 4. Same as Attempt 3, but the controller object in the config is changed to 'con1' and 'con2' respectively.

In [7]:
qmm1 = QuantumMachinesManager()

2021-04-12 21:30:26,037 - qm - INFO - Performing health check
2021-04-12 21:30:26,040 - qm - INFO - Health check passed


In [8]:
# keep controller name in config1 as 'con1'
qm1 = qmm1.open_qm(config1, close_other_machines=False)
qm1_id = qm1.id
print(qm1_id)

qm-1618288224372


In [7]:
# change controller name in config2 to 'con2'
#qm2 = qmm1.open_qm(config2, close_other_machines=False)
#qm2_id = qm2.id
#print(qm2_id)

# this cell throws a runtime error - Failed to contact QM manager

#### Note: Seems like changing the controller to 'con2' caused the runtime error. To confirm this, I will change the controller in config2 to some arbitrary name, e.g. 'aj'. I expect to get the same runtime error.

In [6]:
# change controller name in config2 to 'aj'
#qm2 = qmm1.open_qm(config2, close_other_machines=False)
#qm2_id = qm2.id
#print(qm2_id)
# we get the same runtime error

#### Result: runtime error. Seems like the controller name 'con1' is hard-coded. QM config spec does mention that currently only one controller is supported, so that might be the reason for this runtime error. 

### Attempt 5. I'm getting desperate... maybe we need to open QMs in separate python runtimes??? 

In [5]:
qmm1 = QuantumMachinesManager()

2021-04-12 21:40:46,101 - qm - INFO - Performing health check
2021-04-12 21:40:46,103 - qm - INFO - Health check passed


In [6]:
# open qm1 with this Jupyter notebook on this PC
qm1 = qmm1.open_qm(config1, close_other_machines=False)
qm1_id = qm1.id
print(qm1_id)

qm-1618288844727


In [7]:
# open qm2 via the terminal on this PC (in the same Conda env)
# the following commands were run in the terminal
#qmm = QuantumMachinesManager()
#qmm.list_open_quantum_machines() # this gave the id of the QM opened above, as expected
#qm2 = qmm.open_qm(config2, close_other_machines=False)
#print(qm2.id)
#qmm.list_open_quantum_machines()

#### Result: nope, still not able to fulfill Specific goal. Ok, now I'm going to run the cell above on a different PC that is on the same internet server as the QM server. 

### Attempt 6. Open multiple QMs on different PCs.

In [11]:
qmm = QuantumMachinesManager()

2021-04-13 00:31:24,320 - qm - INFO - Performing health check
2021-04-13 00:31:24,322 - qm - INFO - Health check passed


In [4]:
# open qm1 with config1 with this Jupyter notebook on this PC
qm1 = qmm.open_qm(config1, close_other_machines=False)
qm1_id = qm1.id
print(qm1_id)

qm-1618289317697


In [ ]:
# open qm1 with config2 on a different PC that was on the same internet server as the QM server.
# the following commands were run on the other PC
#qmm = QuantumMachinesManager()
#qmm.list_open_quantum_machines() # this gave the id of the QM opened above, as expected
#qm2 = qmm.open_qm(config2, close_other_machines=False)
#print(qm2.id)
#qmm.list_open_quantum_machines()

#### Result: nope... seems like two distinct quantum machines require two distinct OPXs.... time to ask for help...

In [12]:
qmm.version()

{'client': '0.80.627', 'server': '0.80.627'}